In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['HF_HOME'] = '/shared/data3/pk36/.cache'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
!export HF_HOME=/shared/data3/pk36/.cache

In [3]:
from model_definitions import bertEncode
from taxonomy import Node, Taxonomy
import subprocess
import pickle as pk
from tqdm import tqdm
import numpy as np
from collections import deque
import json
from collections import defaultdict, Counter
from itertools import compress
from nltk.corpus import stopwords
import time
import torch
from scipy import stats
import math
from transformers import BertModel, BertTokenizer
from vllm import LLM, SamplingParams
#from model_definitions import llama_8b_model, sentence_model, promptLlama, constructPrompt, bertEncode
from utils import *
from expansion import llmExpansion
from prompts import *
from main import commonSenseEnrich, computeClassEmb
import sys  
sys.path.insert(1, './lbm')
from prepare_training_data import construct_samples, create_dataset, prepare_data
from model import *
from classifier_training import *
from eval_classifier import *

INFO 11-01 12:12:23 config.py:729] Defaulting to use mp for distributed inference
WARNING 11-01 12:12:23 arg_utils.py:766] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-01 12:12:23 config.py:820] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-01 12:12:23 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False,

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 11-01 12:12:56 model_runner.py:732] Loading model weights took 7.5122 GB
(VllmWorkerProcess pid=3832099) INFO 11-01 12:12:56 model_runner.py:732] Loading model weights took 7.5122 GB
INFO 11-01 12:12:57 distributed_gpu_executor.py:56] # GPU blocks: 15716, # CPU blocks: 4096
INFO 11-01 12:12:59 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-01 12:12:59 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(VllmWorkerProcess pid=3832099) INFO 11-01 12:12:59 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in e

/home/pk36/Comparative-Summarization/taxoadapt/env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
class Args:
    def __init__(self):
        self.dataset = "ner_event_kgc"
        self.data_dir = f"datasets/gen_kgc/{self.dataset}/"
        self.internal = f"{self.dataset}.txt"
        self.external = f"{self.dataset}_external.txt"
        self.groundtruth = "groundtruth.txt"
        
        self.length = 512
        self.dim = 768

        self.iters = 4
        self.model = "bert_full_ft"
        self.override = True
        self.max_depth = 5

        # training classifier
        self.batch_size = 16
        self.epoch = 5
        self.lr = 5e-5

args = Args()

**Reading in input**

In [80]:
# create taxonomy from input
taxo = Taxonomy(args.data_dir)

taxo_dict = taxo.toDict(cur_node=taxo.root)
with open(os.path.join(args.data_dir, 'initial.json'), 'w') as fp:
    json.dump(taxo_dict, fp, indent=4)
dict_str = json.dumps(taxo_dict, indent=4)

In [81]:
taxo.id2label

{'0': 'generative_knowledge_graph_construction',
 '1': 'generation_tasks',
 '2': 'relation_extraction',
 '3': 'entity_linking',
 '4': 'knowledge_graph_completion'}

**LLM Expansion**

In [82]:
expansion_prompts, candidate_nodes = llmExpansion(taxo, sentence_model, temperature=0.3)
taxo_dict = taxo.toDict(cur_node=taxo.root)
with open(os.path.join(args.data_dir, 'expanded.json'), 'w') as fp:
    json.dump(taxo_dict, fp, indent=4)
dict_str = json.dumps(taxo_dict, indent=4)

Processed prompts: 100%|██████████| 5/5 [00:01<00:00,  3.64it/s, est. speed input: 1333.02 toks/s, output: 133.88 toks/s]


In [83]:
candidate_nodes

[{'parent_node': 'generative_knowledge_graph_construction',
  'candidate_nodes': ['graph_construction',
   'knowledge_graph',
   'graph_representation',
   'generative_modeling',
   'graph_learning',
   'graph_generation',
   'graph_representation_learning',
   'graph_construction_methods']},
 {'parent_node': 'generation_tasks',
  'candidate_nodes': ['text_generation',
   'language_modeling',
   'data_to_text_synthesis']},
 {'parent_node': 'relation_extraction',
  'candidate_nodes': ['relation_prediction',
   'relation_inference',
   'relation_identification']},
 {'parent_node': 'entity_linking',
  'candidate_nodes': ['entity_disambiguation',
   'entity_typing',
   'entity_resolution']},
 {'parent_node': 'knowledge_graph_completion',
  'candidate_nodes': ['knowledge_graph_completion_with_context',
   'knowledge_graph_completion_with_graph',
   'knowledge_graph_completion_with_relation']}]

**Common-sense enrichment**

In [15]:
enrich_start = time.time()
prompts, outputs, all_common_phrases, all_common_sentences = commonSenseEnrich(taxo, dict_str, True)
# prompts, output_dict = commonSenseEnrich(taxo.root, dict_str, True)
enrich_end = time.time()

if all_common_phrases:
    # update vocabulary/embeddings
    taxo.updateVocab(all_common_phrases, 'phrases')
    taxo.updateVocab(all_common_sentences, 'sentences')
    print(f"Time taken: {enrich_end - enrich_start} seconds ({(enrich_end - enrich_start)/60} minutes)")

TypeError: cannot unpack non-iterable NoneType object

In [46]:
taxo.label2id

{'generative_knowledge_graph_construction': '0',
 'generation_tasks': '1',
 'relation_extraction': '2',
 'entity_linking': '3',
 'knowledge_graph_completion': '4',
 'knowledge_graph_representation': 5,
 'graph_generation': 6,
 'graph_construction': 7,
 'text_generation': 8,
 'language_modeling': 9,
 'dialog_systems': 10,
 'relation_prediction': 11,
 'entity_discovery': 12,
 'relation_identification': 13,
 'entity_disambiguation': 14,
 'entity_typing': 15,
 'entity_normalization': 16,
 'graph_completion_with_context': 17,
 'knowledge_graph_inference': 18,
 'graph_completion_with_reasoning': 19}

In [7]:
updated_dict = taxo.toDict(cur_node=taxo.root)
with open(os.path.join(args.data_dir, 'enriched.json'), 'w') as fp:
    json.dump(updated_dict, fp, indent=4)

In [8]:
with open("preprocessing/AutoPhrase/data/EN/wiki_quality_orig.txt", "r", encoding='utf-8') as f:
    all_phrases = [w.strip() for w in f.readlines() if " " in w.strip()]
    for a in list(taxo.label2id.keys()) + all_common_phrases:
        all_phrases.append(a.strip().replace("_", " "))

with open("preprocessing/AutoPhrase/data/EN/wiki_quality.txt", "w", encoding='utf-8') as f:
    for w_id, w in enumerate(all_phrases):
        if w_id == (len(all_phrases) - 1):
            f.write(f"{w}")
        else:
            f.write(f"{w}\n")

In [9]:
if args.override:
    # pre-process
    os.chdir("./preprocessing")
    subprocess.check_call(['./auto_phrase.sh', args.data_dir, args.internal])
    subprocess.check_call(['./auto_phrase.sh', args.data_dir, args.external])
    os.chdir("../")
else:
    print("already pre-processed!")

===Corpus Pre-processing===


100%|██████████| 34/34 [00:01<00:00, 19.95it/s]


===Compilation===
===Tokenization===



real	0m1.793s
user	0m13.601s
sys	0m0.897s


Detected Language: EN
Current step: Tokenizing wikipedia phrases...
No provided expert labels.
===Part-Of-Speech Tagging===
Current step: Merging...s...
===AutoPhrasing===


...

=== Current Settings ===
Segmentation Model Path = models/NEW/segmentation.model
After the phrasal segmentation, only following phrases will be highlighted with <phrase> and </phrase>
	Q(multi-word phrases) >= 0.700000
	Q(single-word phrases) >= 1.000000
POS guided model loaded.
# of loaded patterns = 24504
# of loaded truth patterns = 35742
POS transition matrix loaded
Phrasal segmentation finished.
   # of total highlighted quality phrases = 102610
   # of total processed sentences = 189417
   avg highlights per sentence = 0.541715

real	0m2.422s
user	0m2.368s
sys	0m0.040s


===Generating Output===
===Segmented Corpus Post-processing===


4809it [00:00, 24386.00it/s]

Phrase segmented corpus written to ../datasets/gen_kgc/ner_event_kgc//phrase_ner_event_kgc_external.txt


8150it [00:00, 22665.37it/s]


In [10]:
# taxo.static_emb = {}
# taxo.token_lens = {}
# taxo.collection = []
# taxo.external_collection = []

In [11]:
collection, external_collection = taxo.createCollections(args)

Constructing Collections...


100%|██████████| 8184/8184 [00:05<00:00, 1418.06it/s]


**External & Internal Term Enrichment**

In [12]:
print("encoding sentences...")
external_sentences = list(set([sentence for paper in external_collection for sentence in paper.sent_tokenize]))
internal_sentences = list(set([sentence for paper in collection for sentence in paper.sent_tokenize]))
taxo.updateVocab(external_sentences + internal_sentences, 'sentences')
# sent2emb = {sent:idx for idx, sent in enumerate(external_sentences)}
# external_sent_emb = {idx:emb for idx, emb in  enumerate(sentence_model.encode(external_sentences))}

print("encoding phrases...")
external_phrases = list(set([phrase for paper in external_collection for sentence in paper.phrase_tokenize for phrase in sentence]))
internal_phrases = list(set([phrase for paper in collection for sentence in paper.phrase_tokenize for phrase in sentence]))
taxo.updateVocab(external_phrases + internal_phrases, 'phrases')
# phrase2emb = {phrase:idx for idx, phrase in enumerate(external_phrases)}
# external_phrase_emb = {idx:emb for idx, emb in  enumerate(sentence_model.encode(external_phrases))}

taxo.graph.external['phrases'] = external_phrases
taxo.graph.external['sentences'] = external_sentences
taxo.graph.internal['phrases'] = internal_phrases
taxo.graph.internal['sentences'] = internal_sentences

encoding sentences...
encoding phrases...


In [13]:
phrase_pool = external_phrases # list(taxo.vocab['phrases'].keys())
pool_emb = np.array([taxo.vocab['phrases'][w] for w in phrase_pool])

In [14]:
node_external_phrase_ranks, gt, preds = expandDiscriminative(taxo, phrase_pool, pool_emb, internal=False)
f1_scores(gt, preds)

100%|██████████| 5/5 [00:00<00:00, 24105.20it/s]

0.7789915966386554
F1-Macro Score: 0.6585470085470085
F1-Micro Score: 0.773913043478261


In [15]:
term_to_idx, td_matrix, co_matrix = constructTermDocMatrix(taxo, collection + external_collection)
co_avg = np.true_divide(co_matrix.sum(),(co_matrix!=0).sum())
phrase_pool = internal_phrases # list(taxo.vocab['phrases'].keys())
pool_emb = np.array([taxo.vocab['phrases'][w] for w in phrase_pool])

100%|██████████| 8184/8184 [00:15<00:00, 511.90it/s] 


In [16]:
bm_score = computeBM25Cog(co_matrix, co_avg, k=1.2, b=2)

In [17]:
node_internal_phrase_ranks, gt, preds = expandInternal(taxo, phrase_pool, pool_emb, term_to_idx, bm_score)
f1_scores(gt, preds)

100%|██████████| 5/5 [00:01<00:00,  3.85it/s]

0.8910364145658264
F1-Macro Score: 0.6210752688172043
F1-Micro Score: 0.8783068783068783


**Internal Sentence Enrichment**

In [18]:
sentence_pool, full_sent_ranks = expandSentences(taxo, term_to_idx, bm_score)

100%|██████████| 3/3 [00:39<00:00, 13.25s/it]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


**Paper Classification**

In [19]:
class_embs = []
for node_id in tqdm(taxo.id2label):
    curr_node = taxo.root.findChild(node_id)
    curr_node.emb['sentence'] = average_with_harmonic_series(np.stack([taxo.vocab['sentences'][w] 
                                    for w in curr_node.getAllTerms(granularity='sentences', children=True)], axis=0), axis=0)
    class_embs.append(curr_node.emb['sentence'])
    # child.papers = {}
    # child.paper_scores = {}
    # child.ranked = []

100%|██████████| 5/5 [00:00<00:00, 27.39it/s]


In [20]:
corpus = collection + external_collection
preds = [set() for paper in collection + external_collection]
class_map = {i:[] for i in taxo.id2label}
core_classes = {}
all_sent_ranks = []
all_class_weights = []
document_embs = []


for idx, paper in tqdm(enumerate(collection + external_collection), total=len(collection) + len(external_collection)):
    sent_reprs, sent_avg_ranks, sent_dis_ranks = paper.rankSentences(class_embs, phrases=False)
    sent_class_sim = cosine_similarity_embeddings(sent_reprs, class_embs) # .argmax(axis=1) S x C
    weights = np.array(weights_from_ranking([sent_dis_ranks])) # S x 1
    weights /= weights.sum()
    document_embs.append(np.average(sent_reprs, axis=0, weights=weights))
    all_sent_ranks.append(sent_dis_ranks)
    all_class_weights.append(np.average(sent_class_sim, axis=0, weights=weights))
    class_weights = sent_class_sim.T @ weights # C x S @ S x 1
    norm_class_weights = (class_weights - class_weights.min())/(class_weights.max() - class_weights.min())

    # class_weights = np.bincount(sent_to_class, weights=weights, minlength=len(class_embs))
    paper_labels = np.argsort(norm_class_weights)[(np.argmax(np.diff(np.sort(norm_class_weights))) + 1):]
    for i in paper_labels:
        preds[paper.id].add(str(i))
        preds[paper.id] = preds[paper.id].union([p.node_id for p in taxo.root.findChild(str(i), parent=True, node=True)[1]])
        class_map[str(i)].append((paper.id, class_weights[i]))

for k, v in class_map.items():
    score_thresh = np.percentile([s[1] for s in v], 80) if len(v) > 0 else 0
    class_map[k] = sorted([s for s in v if (s[1] >= score_thresh) or (s[0] < len(collection))], key=lambda x: x[1], reverse=True)
    for p in class_map[k]:
        if p[0] in core_classes:
            core_classes[p[0]].append(k)
        else:
            core_classes[p[0]] = [k]

100%|██████████| 8184/8184 [00:03<00:00, 2371.91it/s]


In [21]:
print(example_f1(gt, preds[:len(collection)]))
print(precision_at_k(list(map(list, preds[:len(collection)])), gt, k=1))
print(precision_at_k(list(map(list, preds[:len(collection)])), gt, k=2))
print(precision_at_k(list(map(list, preds[:len(collection)])), gt, k=3))
f1_scores(gt, preds[:len(collection)])

0.8522408963585434
0.7058823529411765
0.6911764705882353
0.7058823529411764
F1-Macro Score: 0.723681592039801
F1-Micro Score: 0.8541666666666666


In [42]:
print(example_f1(gt, preds[:len(collection)]))
print(precision_at_k(list(map(list, preds[:len(collection)])), gt, k=1))
print(precision_at_k(list(map(list, preds[:len(collection)])), gt, k=2))
print(precision_at_k(list(map(list, preds[:len(collection)])), gt, k=3))
f1_scores(gt, preds[:len(collection)])

0.8756302521008403
0.9411764705882353
0.8970588235294118
0.7745098039215688
F1-Macro Score: 0.7437895437895439
F1-Micro Score: 0.8817204301075269


**Expansion**

In [22]:
from transformers import pipeline, T5Tokenizer, T5Config, T5ForConditionalGeneration

In [23]:
T5_PATH = 't5-base' 
DEVICE = torch.device('cuda:' + str(1))
t5_tokenizer = T5Tokenizer.from_pretrained(T5_PATH)
t5_config = T5Config.from_pretrained(T5_PATH)
t5_mlm = T5ForConditionalGeneration.from_pretrained(T5_PATH, config=t5_config).to(DEVICE)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/pk36/Comparative-Summarization/taxoadapt/env/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [98]:
from model_definitions import constructPrompt, promptLlamaVLLM
from prompts import SiblingSchema

In [131]:
def widthExpansion(node, unmapped_papers):
    prompts = []

    for p_id in unmapped_papers:
        # get common-sense & corpus-specific options
        init_prompt = 'You are an assistant that performs width expansion of taxonomies. Width expansion in taxonomies adds more categories at the same level, increasing options without adding depth. For example, expanding a taxonomy of NLP tasks from [\"text_classification\" and \"named_entity_recognition\"] to include \"sentiment_analysis\", \"machine_translation\", and \"question_answering\" would be a width expansion.'
        # The content of this paper is "{collection[p_id].content}"\n\n
        main_prompt = f'The title of the paper is "{collection[p_id].title}".\n\nThe abstract of this paper is "{collection[p_id].abstract}"\n\nParents/Ancestor Topics: {node.path[1:]}\n\nSibling Topics: {", ".join(map(str, node.parents[0].children))}.\n\nBased on the given paper, can you expand the set of siblings with several options? Output your answer as a Python list only containing only the new siblings.\n\nOutput Format:\n\n{{"new_siblings": <list of strings where values are the new siblings (1-3 words) based on the given paper and existing siblings>}}'

        prompts.append(constructPrompt(init_prompt, main_prompt, api=False))

    output_dict = promptLlamaVLLM(prompts, schema=SiblingSchema, max_new_tokens=500)
    output = [json.loads(clean_json_string(c)) if "```json" in c else json.loads(c.strip()) for c in output_dict]

    all_candidates = []
    for c in output:
        all_candidates.extend(c['new_siblings'])

    return output, all_candidates

In [132]:
w_output, w_candidates = widthExpansion(taxo.root.children[0], unmapped_papers)

Processed prompts: 100%|██████████| 8/8 [00:02<00:00,  2.81it/s, est. speed input: 1111.62 toks/s, output: 100.83 toks/s]


In [109]:
unmapped_papers

[1, 7, 12, 13, 18, 22, 24, 31]

In [117]:
candidate_freq = sorted(dict(Counter(w_candidates)).items(), key=lambda x: -x[1])

In [133]:
w_candidates

['discriminative_methods',
 'label_semantics',
 'few_shot_tasks',
 'high_resource_tasks',
 'low_resource_tasks',
 'bert_baseline',
 'snips_dataset',
 'text_to_structure',
 'schema_based',
 'large_scale',
 'pretrained_model',
 'state_of_the_art',
 'unified_framework',
 'universal_model',
 'adaptive_generation',
 'collaborative_learning',
 'general_abilities',
 'information_extraction',
 'sequence_generation',
 'knowledge_graph_construction',
 'natural_language_processing',
 'triplet_contrastive_training',
 'batch_wise_dynamic_attention_masking',
 'triple_wise_calibration',
 'encoder_decoder_based_generation',
 'generative_transformer',
 'contrastive_information_extraction',
 'contrastive_learning',
 'triplet_extraction',
 'knowledge_graph',
 'text_generation',
 'language_translation',
 'machine_reading',
 'event_centric_reasoning',
 'correlation_aware_transformer',
 'event_correlation_encoding',
 'zero_shot_cross_lingual_transfer',
 'cross_lingual_event_argument_extraction',
 'cross_lin

In [118]:
candidate_freq

[('information_extraction', 4),
 ('sequence_generation', 3),
 ('sequence_to_sequence', 2),
 ('sequence_understanding', 2),
 ('sequence_representation', 2),
 ('event_extraction', 2),
 ('natural_language_processing', 2),
 ('sequence_labeling', 1),
 ('sequence_tagging', 1),
 ('sequence_prediction', 1),
 ('sequence_classification', 1),
 ('sequence_learning', 1),
 ('sequence_modeling', 1),
 ('text_to_structure', 1),
 ('universal_model', 1),
 ('structured_representation', 1),
 ('schema_based_prompt', 1),
 ('structured_extraction_language', 1),
 ('text_to_structure_generation', 1),
 ('structured_schema_instructor', 1),
 ('structured_language_model', 1),
 ('structured_records', 1),
 ('relation_extraction', 1),
 ('named_entity_recognition', 1),
 ('relation_classification', 1),
 ('knowledge_graph_construction', 1),
 ('language_understanding', 1),
 ('language_generation', 1),
 ('encoder_decoder_architecture', 1),
 ('natural_language_generation', 1),
 ('sequence_to_text', 1),
 ('text_to_text', 1),

In [128]:
len(w_output)

8

In [130]:
", ".join(w_output[6]['new_siblings'])

'zero_shot_cross_lingual_transfer, cross_lingual_event_extraction, cross_lingual_relation_extraction, cross_lingual_dependency_parsing, cross_lingual_named_entity_recognition, cross_lingual_relation_labeling, cross_lingual_event_argument_extraction, cross_lingual_transfer_learning, cross_lingual_language_models, cross_lingual_nlp_tasks'

In [ ]:
queue = deque([taxo.root])
unmapped_papers = {}
expansion_candidates = {}

while queue:
    curr_node = queue.popleft()
    path_ids = {taxo.label2id[i] for i in curr_node.path[1:]} # ignore root

    # width expansion (0): if the current node is not a leaf node, get the nodes which are mapped to it, but not any of its children
    # depth expansion (1): if the current node is a leaf node, get the nodes which are mapped to it
    unmapped_papers[curr_node.node_id] = [p_id for p_id, p in enumerate(collection) if preds[p_id] == path_ids]

    if len(curr_node.children):
        w_output, w_candidates = widthExpansion(curr_node, unmapped_papers[curr_node.node_id])
    else:
        depthExpansion(curr_node, unmapped_papers[curr_node.node_id])

In [25]:
unmapped_papers = []
mapped_papers = []

for idx, p in enumerate(zip(gt, preds[:len(collection)], all_class_weights)):
    if sorted(p[1]) == ['0', '1']:
        unmapped_papers.append(idx)
        print(f'paper_id: {idx}; preds: {sorted(p[1])}; gt: {p[0]}; weights: {p[2]}')
        print(f'paper title: {collection[idx].title}')
    else:
        mapped_papers.append(idx)

paper_id: 1; preds: ['0', '1']; gt: ['0', '1']; weights: [0.84531204 0.86006769 0.80433236 0.78352513 0.80453067]
paper title: augmented_natural_language for generative sequence_labeling
paper_id: 7; preds: ['0', '1']; gt: ['0', '1', '2']; weights: [0.82464512 0.84528683 0.7948382  0.7818779  0.79149367]
paper title: unified structure_generation for universal information_extraction
paper_id: 12; preds: ['0', '1']; gt: ['0', '1', '2']; weights: [0.82684119 0.83132468 0.79769621 0.77207515 0.79802967]
paper title: contrastive information_extraction with generative_transformer
paper_id: 13; preds: ['0', '1']; gt: ['0', '1', '2']; weights: [0.82035241 0.84272177 0.78811188 0.76300622 0.78246604]
paper title: contrastive triple_extraction with generative_transformer
paper_id: 18; preds: ['0', '1']; gt: ['0', '1']; weights: [0.82936326 0.83146916 0.79841426 0.79143692 0.79385458]
paper title: template filling with generative transformers
paper_id: 22; preds: ['0', '1']; gt: ['0', '1', '4']; 

In [25]:
template = f'The title of the paper is "{collection[unmapped_papers[0]].title}". The abstract of this paper is "{collection[unmapped_papers[4]].abstract}"\n\n According to the paper, its specific 1-3 word category is "<extra_id_0>". Other examples of adjacent, sibling categories to the paper\'s category are: {", ".join(map(str, taxo.root.children[0].children))}.'
print(template)

The title of the paper is "template filling with generative transformers". The abstract of this paper is "template filling is generally tackled by a pipeline of two separate supervised systems one for role_filler extraction and another for template/event recognition . since pipelines consider events in isolation , they can suffer from error_propagation . we introduce a framework based on end_to_end generative transformers for this task ( i.e. , gtt ) . it naturally models the dependence between entities both within a single event and across the multiple events described in a document . experiments demonstrate that this framework substantially outperforms pipeline_based approaches , and other neural end_to_end baselines that do not model between_event dependencies . we further show that our framework specifically improves performance on documents containing multiple events ."

 According to the paper, its specific 1-3 word category is "<extra_id_0>". Other examples of adjacent, sibling 

In [26]:
temp_res = []
end_token='<extra_id_1>'

encoded = t5_tokenizer.encode_plus(template, add_special_tokens=True, truncation=True, return_tensors='pt')
input_ids = encoded['input_ids'].to(DEVICE)
outputs = t5_mlm.generate(input_ids=input_ids, 
                            num_beams=100, num_return_sequences=10,
                            max_length=10)

for output in outputs:
    _txt = t5_tokenizer.decode(output[2:], skip_special_tokens=False, clean_up_tokenization_spaces=False)
    if end_token in _txt:
        _end_token_index = _txt.index(end_token)
        temp_res.append(_txt[:_end_token_index].strip())
temp_res

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['template_filling',
 'template filling',
 'template filling',
 'documents containing multiple events',
 'document_filling']

In [72]:
template = f'The title of the paper is "{collection[1].title}". The abstract of this paper is "{collection[1].abstract}"\n\nThis paper has already been classified under the parent topics: {taxo.root.children[0].path[1:]}.\n\nBased on the paper\'s title and abstract, from the adjacent category options, [ {", ".join(map(str, taxo.root.children[0].children))}, and <extra_id_0> ], this paper falls under category option " <extra_id_0> ". Fill in the mask, <extra_id_0>. Only output a Python list of 10 options.'

In [73]:
print(template)

The title of the paper is "augmented_natural_language for generative sequence_labeling". The abstract of this paper is "we propose a generative_framework for joint sequence_labeling and sentence_level classification . our model performs multiple sequence_labeling tasks at once using a single , shared natural_language output space . unlike prior discriminative methods , our model naturally incorporates label_semantics and shares knowledge across tasks . our framework is general purpose , performing well on few_shot , low_resource , and high_resource tasks . we demonstrate these advantages on popular named_entity_recognition , slot_labeling , and intent classification benchmarks . we set a new state_of_the_art for few_shot slot_labeling , improving substantially upon the previous 5_shot ( 75.0 % 90.9 % ) and 1_shot ( 70.4 % 81.0 % ) state_of_the_art results . furthermore , our model generates large improvements ( 46.27 % 63.83 % ) in low_resource slot_labeling over a bert baseline by inc

In [80]:
res = []
all_res = []
end_token='<extra_id_1>'

for p_id in unmapped_papers:
    # template = f'The title of the paper is "{collection[p_id].title}". The abstract of this paper is "{collection[p_id].abstract}"\n\n According to the paper title and abstract, the specific 1-3 word category of the paper is " <extra_id_0> ".\n\nIn the category taxonomy, the path to " <extra_id_0> " is: {" -> ".join(taxo.root.children[0].path[1:])} -> <extra_id_0>. Other examples of adjacent, sibling categories to the paper\'s category, <extra_id_0> , are: {", ".join(map(str, taxo.root.children[0].children))}.'
    template = f'The title of the paper is "{collection[p_id].title}". The abstract of this paper is "{collection[p_id].abstract}"\n\nBased on the paper\'s title and abstract, from the adjacent category options, [ {", ".join(map(str, taxo.root.children[0].children))}, and <extra_id_0> ], this paper falls under category option " <extra_id_0> ".'

    encoded = t5_tokenizer.encode_plus(template, add_special_tokens=True, truncation=True, return_tensors='pt')
    input_ids = encoded['input_ids'].to(DEVICE)
    outputs = t5_mlm.generate(input_ids=input_ids, 
                                num_beams=20, num_return_sequences=10,
                                max_length=10)

    for output in outputs:
        _txt = t5_tokenizer.decode(output[2:], skip_special_tokens=False, clean_up_tokenization_spaces=False)
        if end_token in _txt:
            _end_token_index = _txt.index(end_token)
            res.append(_txt[:_end_token_index].strip().lower().replace(' ', '_'))
            all_res.append(_txt[:_end_token_index].strip().lower().replace(' ', '_'))
        else:
            all_res.append(_txt)

In [81]:
set(res)

{'.',
 'augmented_natural_language',
 'augmented_natural_language.',
 'claret',
 'comparison_learning',
 'contrast_learning',
 'contrastive_learning',
 'entity_extraction',
 'entity_linking',
 'event_extraction',
 'generative_language_models',
 'generative_transformer',
 'generative_transformer.',
 'generative_transformers',
 'graph_generation',
 'ie_extraction',
 'information_extraction',
 'information_extraction.',
 'knowledge_graph_building',
 'knowledge_graph_completion',
 'knowledge_graph_construction',
 'relation_extraction',
 'semantic_extraction',
 'semantics_extraction',
 'template_filling',
 'template_filling.',
 'text_to_structure',
 'unified_structure_generation'}

In [58]:
set(all_res)

{'.',
 'Template filling with generative transformers',
 'augmented_natural_language',
 'augmented_natural_language for',
 'augmented_natural_language.',
 'claret',
 'comparison_learning',
 'contrast_learning',
 'contrastive_learning',
 'correlation_aware context_to_',
 'entity_extraction',
 'entity_linking',
 'event_extraction',
 'generative language_models for zero',
 'generative_language_models',
 'generative_transformer',
 'generative_transformer.',
 'generative_transformers',
 'graph_generation',
 'graph_generation : an empirical study',
 'ie_extraction',
 'information_extraction',
 'information_extraction with generative_',
 'information_extraction.',
 'knowledge_graph_building',
 'knowledge_graph_completion',
 'knowledge_graph_construction',
 'multilingual generative language_models',
 'relation_extraction',
 'semantic_extraction',
 'semantics_extraction',
 'template filling with generative transformer',
 'template_filling',
 'template_filling with generative',
 'template_fillin

In [125]:
set(res)

{'Prompt for extraction ?',
 'claret',
 'data_extraction',
 'deepstruct',
 'document_filling',
 'documents containing multiple events',
 'dynamic prefix_tuning',
 'education_extraction',
 'event_centric',
 'event_centric generation and classification',
 'event_extraction',
 'explanation_graph',
 'explanation_graph_generation',
 'expression_extraction',
 'extraction',
 'generative event_extraction',
 'generic event_extraction',
 'human_annotated graphs',
 'information_extraction',
 'information_extraction .',
 'information_extraction.',
 'intellectual_extraction',
 'intelligent_extraction',
 'intent_recognition',
 'knowledge_graph_completion',
 'learning_graph_completion',
 'no_shot cross_lingual',
 'object_extraction',
 'prompt for extraction',
 'prompt for extraction ?',
 'prompt for extractive objectives',
 'relation_extraction',
 'sequence_extraction',
 'structure_prediction',
 'template filling',
 'template_filling'}

In [ ]:
# identify the phrases in unmapped papers that are:
## how freq in 
unmapped_phrase_pool = {}
for p_id in unmapped_papers:
    collection[p_id].vocabulary

In [49]:
# other clusters
node_to_expand = '1'
top_k = 3
parent_node = taxo.root.findChild(node_to_expand)
cluster_nodes = parent_node.children

print(f"Each paper in a corpus is either assigned to a cluster (mapped) or not (unmapped). Given the following existing clusters and their respective mapped papers below (tag 'existing_clusters') and the list of unmapped papers (tag 'unmapped_papers'), first determine the unmapped papers that SHOULD NOT BE MAPPED to an existing_cluster. From this list of unmapped and irrelevant to existing clusters papers, determine HOW MANY clusters are formed from the list, and what the cluster topic(s) should be. The constraints for cluster topic titles are that (1) they must be a subtopic of parent topic {parent_node.label}, (2) easily be replaced with (a sibling to) any of the existing cluster titles in the statement, \"[cluster title] is a subtopic of {parent_node.label}\", and (3) at least 2 papers must be mapped to the cluster. If no clusters should be formed (all unmapped papers should truly be mapped to an existing cluster or less than two papers is mapped to the potential cluster), then simply output 0 for 'number_of_clusters'.")

print("\nexisting_clusters:")
for c_id, c in enumerate(cluster_nodes):
    print(f'Cluster {c_id}: {c.label} ({c.description})')
    # cluster_papers = class_map[c.node_id][:top_k]
    # for p in cluster_papers:
    #     print(f'- PAPER ID: {corpus[p[0]].id}; {corpus[p[0]].title}: {corpus[p[0]].abstract}')
    # print("\n")

print("\nunmapped_papers:")
for p in no_label_papers:
    print(f'- PAPER ID: {corpus[p].id}; {corpus[p].title}: {corpus[p].abstract}')

print("""\nOutput your answer in the following JSON format: 
      {
      "number_of_clusters": <integer: number of new k clusters with at least 2 papers mapped to each>,
      "new_cluster_1": {
            "title": <string value of cluster_1 title; 2-3 words, lowercase, underscore instead of space>
            "mapped_papers": <list of at least 2 paper ids which should be mapped to this new cluster_1>
        }
      ...
      "new_cluster_k": {
            "title": <string value of cluster_k title; 2-3 words, lowercase, underscore instead of space>
            "mapped_papers": <list of at least 2 paper ids which should be mapped to this new cluster_k>
        }
      }
      """)


Each paper in a corpus is either assigned to a cluster (mapped) or not (unmapped). Given the following existing clusters and their respective mapped papers below (tag 'existing_clusters') and the list of unmapped papers (tag 'unmapped_papers'), first determine the unmapped papers that SHOULD NOT BE MAPPED to an existing_cluster. From this list of unmapped and irrelevant to existing clusters papers, determine HOW MANY clusters are formed from the list, and what the cluster topic(s) should be. The constraints for cluster topic titles are that (1) they must be a subtopic of parent topic generation_tasks, (2) easily be replaced with (a sibling to) any of the existing cluster titles in the statement, "[cluster title] is a subtopic of generation_tasks", and (3) at least 2 papers must be mapped to the cluster. If no clusters should be formed (all unmapped papers should truly be mapped to an existing cluster or less than two papers is mapped to the potential cluster), then simply output 0 for 

In [28]:
taxo.id2label

{'0': 'generative_knowledge_graph_construction',
 '1': 'generation_tasks',
 '2': 'relation_extraction',
 '3': 'entity_linking',
 '4': 'knowledge_graph_completion'}

In [48]:
for idx, p in enumerate(preds[:len(collection)]):
    print(idx+1, p)

1 {'0', '1', '3'}
2 {'0', '1'}
3 {'0', '1', '2'}
4 {'0', '1', '2'}
5 {'0', '1', '3'}
6 {'0'}
7 {'0', '1', '3', '2'}
8 {'0', '1'}
9 {'0', '1', '2'}
10 {'0', '1', '2'}
11 {'0', '1', '2'}
12 {'0', '1', '3', '2'}
13 {'0', '1', '2'}
14 {'0', '1'}
15 {'0', '1', '3', '2'}
16 {'0', '1', '3', '2'}
17 {'0', '1', '3', '2'}
18 {'0', '1', '3', '2'}
19 {'0', '1', '2'}
20 {'0', '1', '3'}
21 {'0', '1', '2'}
22 {'0', '1', '2'}
23 {'0', '1', '2'}
24 {'0', '1', '2'}
25 {'0', '1'}
26 {'0', '1', '2'}
27 {'0', '1', '3'}
28 {'0', '1', '3'}
29 {'0', '1', '2', '4'}
30 {'0', '1', '4'}
31 {'0', '1', '4'}
32 {'0', '1', '2', '4'}
33 {'0', '1', '4'}
34 {'0', '1', '4'}


In [23]:
from model_definitions import promptGPT, constructPrompt

In [24]:
def generate_summary_title(parent, paper_list, related_titles_list):
    # Prompt engineering to provide context and desired output
    prompt = f"""You are given a group of paper titles/abstracts and a list of sibling topics. Your task is to generate a topic keyword that summarizes the cluster of the given papers.
    The topic should be a subtopic of {parent} and at the same topic granularity as the provided list of sibling topic titles. Here are the papers in the cluster:"""

    prompt += "\n\n".join([f"- {abstract}" for abstract in paper_list])
    prompt += "\n\nHere are the sibling topic titles:\n"
    prompt += "\n".join([f"- {title}" for title in related_titles_list])
    prompt += "\n\nBased on these papers and siblings, please first consider 5 candidate topics that represents ALL of the papers within the cluster. Then generate a topic title that summarizes these 5 topics and output it in the same format as the sibling topics (lowercase, underscores instead of spaces)."

    prompts = [constructPrompt("You are an expert summarizer and topic generator for scientific research.", prompt, api=True)]
    output = promptGPT(prompts, schema=None, max_new_tokens=200, json_mode=False)
    
    # Extract and return the title from the response
    return output

In [25]:
generated_titles = generate_summary_title(
    parent=taxo.root.children[0].label,
    paper_list=[f"{collection[i].title}: {collection[i].abstract}" for i in no_label_papers],
    related_titles_list=[c.label for c in taxo.root.children[0].children]
    )

100%|██████████| 1/1 [00:01<00:00,  1.16s/it]


In [26]:
print(generated_titles[0])

Candidate topics:
- generative_sequence_labeling
- information_extraction_generation
- event_centric_reasoning
- template_based_event_extraction
- cross_lingual_event_argument_extraction

Summary topic title: event_generation_and_extraction


In [ ]:
prompt = f"""You are given a group of paper titles/abstracts and a list of sibling topics. Your task is to generate a topic keyword that summarizes the cluster of the given papers.
    The topic should be a subtopic of {parent} and at the same topic granularity as the provided list of sibling topic titles. Here are the papers in the cluster:"""

prompt += "\n\n".join([f"- {abstract}" for abstract in paper_list])
prompt += "\n\nHere are the sibling topic titles:\n"
prompt += "\n".join([f"- {title}" for title in related_titles_list])
prompt += "\n\nBased on these papers and siblings, please first consider 5 candidate topics that represents ALL of the papers within the cluster. Then generate a topic title that summarizes these 5 topics and output it in the same format as the sibling topics (lowercase, underscores instead of spaces)."

In [209]:
focus_paper = 28
ordered_ranks = np.array([all_sent_ranks[focus_paper][i] for i in np.arange(len(all_sent_ranks[focus_paper]))])
sent_rank = {}
# for s_id, sent in enumerate(external_collection[focus_paper - len(collection)].sent_tokenize):
for s_id, sent in enumerate(collection[focus_paper].sent_tokenize):
    print(s_id, all_sent_ranks[focus_paper][s_id], sent)

# print('title + abstract:', ordered_ranks[:13].mean())
# print('introduction:', ordered_ranks[13:24].mean())
# print('related works:', ordered_ranks[24:31].mean())
# print('datasets:', ordered_ranks[31:43].mean())
# print('methods:', ordered_ranks[43:67].mean())
# print('results:', ordered_ranks[67:80].mean())
# print('conclusion:', ordered_ranks[80:83].mean())

0 1 paper_title : comet : commonsense transformers for automatic knowledge_graph construction ; paper_abstract : we present the first comprehensive study on automatic knowledge_base construction for two prevalent commonsense knowledge graphs : atomic ( sap et al. , 2019 ) and conceptnet ( speer et al. , 2017 )
1 127 contrary to many conventional kbs that store knowledge with canonical templates , commonsense kbs only store loosely structured open_text descriptions of knowledge
2 92 we posit that an important step toward automatic commonsense completion is the development of generative models of commonsense knowledge , and propose commonsense transformers ( comet ) that learn to generate rich and diverse commonsense descriptions in natural_language
3 105 despite the challenges of commonsense modeling , our investigation reveals promising results when implicit knowledge from deep pre_trained_language_models is transferred to generate explicit_knowledge in commonsense knowledge graphs
4 1

**Training**

In [27]:
# core_classes = {}
gt_labels = []

for idx, paper in enumerate(collection):
    # core_classes[paper.id] = preds[idx]
    gt_labels.append(list(map(int, paper.gold)))

# for idx, paper in enumerate(external_collection):
#     core_classes[paper.id] = preds[idx]

if not os.path.exists(os.path.join(args.data_dir, 'train')):
    os.makedirs(os.path.join(args.data_dir, 'train'))
if not os.path.exists(os.path.join(args.data_dir, 'test')):
    os.makedirs(os.path.join(args.data_dir, 'test'))

# json.dump(core_classes, open(os.path.join(args.data_dir, 'train/refined_core_classes.json'), 'w'), indent=1)

In [28]:
from model_definitions import bert_model, bert_tokenizer

In [29]:
args.epoch = 5
args.batch_size = 64

In [30]:
doc2parents, doc2child = prepare_data(args, taxo, core_classes, corpus, bert_model, bert_tokenizer)

Converting texts into tensors.


/home/pk36/Comparative-Summarization/taxoadapt/env/lib/python3.8/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.Bytes

Saving encoded texts into datasets/gen_kgc/ner_event_kgc/train/training_data.pt


In [31]:
train_classifier(args)

/home/pk36/Comparative-Summarization/taxoadapt/./lbm/classifier_training.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_dict = torch.load(os.path.join(args.data_d

Training epoch: 0


  0%|          | 0/42 [00:00<?, ?it/s]/home/pk36/Comparative-Summarization/taxoadapt/./lbm/model.py:132: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha = 1) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1581.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)
100%|██████████| 42/42 [00:48<00:00,  1.16s/it]


Epoch 0 82.41540959986244
Training epoch: 1


100%|██████████| 42/42 [00:49<00:00,  1.17s/it]


...

Epoch 2 14.28550218954319
Training epoch: 3


100%|██████████| 42/42 [00:49<00:00,  1.18s/it]


Epoch 3 8.051641978141738
Training epoch: 4


100%|██████████| 42/42 [00:49<00:00,  1.19s/it]


Epoch 4 6.948110898093479


In [32]:
# 
predictions, top_classes, all_classes, selected_classes = evaluate_cls(args, collection, bert_tokenizer, os.path.join(args.data_dir, f'train/model.pt'), gt_labels)

/home/pk36/Comparative-Summarization/taxoadapt/./lbm/eval_classifier.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  class_emb = torch.load(os.path.join(args.data_dir, 

Converting texts into tensors.


100%|██████████| 1/1 [00:00<00:00,  4.47it/s]

Precision@1: 0.8235294117647058
Precision@2: 0.8676470588235294
Precision@3: 0.843137254901961
MRR: 0.3542592592592593
Example F1 (TOP 3): 0.8901960784313725
Example F1 (SELECTED): 0.865266106442577
Example F1 (ALL): 0.6869747899159663


In [64]:
# 
predictions, top_classes, all_classes, selected_classes = evaluate_cls(args, collection, bert_tokenizer, os.path.join(args.data_dir, f'train/model.pt'), gt_labels)

/home/pk36/Comparative-Summarization/taxoadapt/./lbm/eval_classifier.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  class_emb = torch.load(os.path.join(args.data_dir, 

Converting texts into tensors.


/home/pk36/Comparative-Summarization/taxoadapt/./lbm/eval_classifier.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  class_model.load_state_dict(torch.load(model_pth, m

Precision@1: 1.0
Precision@2: 1.0
Precision@3: 0.9215686274509803
MRR: 0.3379889455782313
Example F1 (TOP 3): 0.890406162464986
Example F1 (SELECTED): 0.8169340463458111
Example F1 (ALL): 0.6355614973262034


In [35]:
f1_scores(gt_labels, top_classes)

F1-Macro Score: 0.5488696488696488
F1-Micro Score: 0.6355140186915887


In [47]:
# 
predictions, top_classes, all_classes, selected_classes = evaluate_cls(args, collection, bert_tokenizer, os.path.join(args.data_dir, f'train/model.pt'), gt_labels)

/home/pk36/Comparative-Summarization/taxoadapt/./lbm/eval_classifier.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  class_emb = torch.load(os.path.join(args.data_dir, 

Converting texts into tensors.


100%|██████████| 3/3 [00:00<00:00, 11.07it/s]

Precision@1: 1.0
Precision@2: 1.0
Precision@3: 0.823529411764706
MRR: 0.3535471331389699
Example F1 (TOP 3): 0.8394957983193276
Example F1 (SELECTED): 0.8745098039215686
Example F1 (ALL): 0.6431372549019608


In [54]:
# rRECENT
predictions, top_classes, all_classes, selected_classes = evaluate_cls(args, collection, bert_tokenizer, os.path.join(args.data_dir, f'train/model.pt'), gt_labels)

Converting texts into tensors.


100%|██████████| 3/3 [00:00<00:00, 10.73it/s]

Precision@1: 1.0
Precision@2: 0.9705882352941176
Precision@3: 0.8529411764705884
MRR: 0.35604956268221577
Example F1 (TOP 3): 0.8675070028011204
Example F1 (SELECTED): 0.8389589169000933
Example F1 (ALL): 0.6431372549019608


In [34]:
predictions, top_classes, all_classes, selected_classes = evaluate_cls(args, collection, bert_tokenizer, os.path.join(args.data_dir, f'train/model.pt'), gt_labels)

/home/pk36/Comparative-Summarization/taxoadapt/./lbm/eval_classifier.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  class_emb = torch.load(os.path.join(args.data_dir, 

Converting texts into tensors.


100%|██████████| 3/3 [00:00<00:00, 11.49it/s]

Precision@1: 1.0
Precision@2: 1.0
Precision@3: 0.7941176470588235
MRR: 0.3519193391642372
Example F1 (TOP 3): 0.8128851540616245
Example F1 (SELECTED): 0.6431372549019608
Example F1 (ALL): 0.807282913165266


In [86]:
for p in zip(gt, selected_classes, predictions):
    print(f'preds: {sorted(p[1])}; gt: {p[0]}; weights: {p[2][sorted(p[1])]}')

preds: [0, 1, 2, 4]; gt: ['0', '1', '2']; weights: [10.774901  11.105757   7.7327485  4.756489 ]
preds: [0, 1, 2]; gt: ['0', '1', '2']; weights: [12.038707 11.597884 10.183065]
preds: [0, 1, 2, 3, 4]; gt: ['0', '1', '2', '3']; weights: [9.584637  9.684874  7.575132  5.1706114 6.488943 ]
preds: [0, 1]; gt: ['0', '1', '2', '3']; weights: [9.953376  9.8585205]
preds: [0, 1, 2, 4]; gt: ['0', '1', '2']; weights: [10.590563 11.031084  8.630755  5.828255]
preds: [0, 1, 2]; gt: ['0', '1', '2', '3']; weights: [11.434895 11.057161  8.91286 ]
preds: [0, 1, 2]; gt: ['0', '1', '2']; weights: [11.662855 11.989753 10.723114]
preds: [0, 1, 2]; gt: ['0', '1', '2', '3']; weights: [10.98599  10.875311  9.590318]
preds: [0, 1, 3]; gt: ['0', '1', '3']; weights: [10.4102   10.507949  6.939367]
preds: [0, 1, 3]; gt: ['0', '1', '3']; weights: [10.551313 11.315516 12.889475]
preds: [0, 1, 3]; gt: ['0', '1', '3']; weights: [ 9.139808  9.618616 11.839673]
preds: [0, 1]; gt: ['0', '1', '3']; weights: [11.577571 1

**Clustering of Papers**

In [31]:
from hdbscan import HDBSCAN
from hdbscan.flat import (HDBSCAN_flat,
                          approximate_predict_flat,
                          membership_vector_flat,
                          all_points_membership_vectors_flat)

In [42]:
document_embs = np.array(document_embs)
selected_document_embs = document_embs[no_label_papers]

In [46]:
clusterer = HDBSCAN(min_cluster_size=2)
clusterer.fit(document_embs[:len(collection)])
cluster_labels = clusterer.labels_
proba = clusterer.probabilities_

In [47]:
for i, (g, c) in enumerate(zip(taxo.gold_labels, cluster_labels)):
    print(i, g[2:], c)

0 [] 0
1 [] 0
2 ['2'] 0
3 ['2'] 0
4 [] 0
5 ['2'] 0
6 [] -1
7 ['2'] -1
8 ['2'] 0
9 ['2'] 0
10 ['2'] 0
11 ['2'] 0
12 ['2'] 0
13 ['2'] 0
14 ['2'] 0
15 ['2'] 0
16 ['2'] 0
17 [] -1
18 [] 0
19 [] 0
20 [] 0
21 [] 0
22 ['4'] 0
23 [] 0
24 [] 0
25 [] 0
26 ['3'] 0
27 ['3'] -1
28 ['4'] 0
29 ['4'] 1
30 ['4'] 1
31 ['4'] 0
32 ['4'] 1
33 ['4'] 1


In [211]:
clusterer = HDBSCAN_flat(document_embs,
                         cluster_selection_method='eom',
                         n_clusters=4, min_cluster_size=2)
cluster_labels = clusterer.labels_
proba = clusterer.probabilities_

In [94]:
taxo.id2label

{'0': 'generative_knowledge_graph_construction',
 '1': 'generation_tasks',
 '2': 'named_entity_recognition',
 '3': 'relation_extraction',
 '4': 'entity_linking',
 '5': 'knowledge_graph_completion'}

In [ ]:
psuedo_labels = {} # node_id: paper
thresh = 0.5

queue = deque([taxo.root])

while queue:
    curr_node = queue.popleft()
    class_embs = []
    for child in tqdm(curr_node.children):
        child.emb['sentence'] = average_with_harmonic_series(np.stack([taxo.vocab['sentences'][w] 
                                        for w in child.getAllTerms(granularity='sentences', children=False)], axis=0), axis=0)
        class_embs.append(child.emb['sentence'])
        child.papers = {}
        child.paper_scores = {}
        child.ranked = []
    
    if len(class_embs):
        # rank sentences based on semantic sim
        # compute paper embedding based discriminative sentence weights
        for paper in collection:
            sent_reprs, sent_avg_ranks, sent_dis_ranks = paper.rankSentences(class_embs, phrases=False)
            sent_to_class = cosine_similarity_embeddings(sent_reprs, class_embs).argmax(axis=1)
            weights = weights_from_ranking([sent_dis_ranks])

            class_weights = np.bincount(sent_to_class, weights=weights, minlength=len(class_embs))
            
            

        # take top 50% of assigned papers as pseudo-labels
        for child in curr_node.children:
            print(len(child.ranked))
            num_papers = int(len(child.ranked) * thresh)
            psuedo_labels[child.node_id] = [collection[p_id] for p_id in child.ranked[:num_papers]]

            queue.append(child)


In [86]:
psuedo_labels = {} # node_id: paper
thresh = 0.5

queue = deque([taxo.root])

while queue:
    curr_node = queue.popleft()
    class_embs = []
    for child in tqdm(curr_node.children):
        child.emb['sentence'] = average_with_harmonic_series(np.stack([taxo.vocab['sentences'][w] 
                                        for w in child.getAllTerms(granularity='sentences', children=False)], axis=0), axis=0)
        class_embs.append(child.emb['sentence'])
        child.papers = {}
        child.paper_scores = {}
        child.ranked = []
    
    if len(class_embs):
        # rank sentences based on semantic sim
        # compute paper embedding based discriminative sentence weights
        paper_embs = np.array([paper.computePaperEmb(class_embs, phrase=False) for paper in collection])

        # for each node, assign papers based on similarity gap
        class_labels, class_paper_scores, mapping = taxo.mapPapers(collection, paper_embs, curr_node.children, class_embs)

        # take top 50% of assigned papers as pseudo-labels
        for child in curr_node.children:
            print(len(child.ranked))
            num_papers = int(len(child.ranked) * thresh)
            psuedo_labels[child.node_id] = [collection[p_id] for p_id in child.ranked[:num_papers]]

            queue.append(child)


100%|██████████| 1/1 [00:00<00:00, 43.72it/s]


34


100%|██████████| 5/5 [00:00<00:00, 81.30it/s]


21
4
11
1
6


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [164]:
args.data_dir

'datasets/gen_kgc/'

In [158]:
preds

[['0', '1', '2'],
 ['0', '1', '2'],
 ['0', '1', '2'],
 ['0', '1', '2'],
 ['0', '1', '2'],
 ['0', '1', '6', '2'],
 ['0', '1', '2'],
 ['0', '1', '2'],
 ['0', '1', '2'],
 ['0', '1', '2', '3'],
 ['0', '1', '2'],
 ['0', '1', '2'],
 ['0', '1', '6', '4', '2'],
 ['0', '1', '2'],
 ['0', '1', '2'],
 ['0', '1', '2', '3'],
 ['0', '1', '2'],
 ['0', '1', '4'],
 ['0', '1', '4'],
 ['0', '1', '2', '4'],
 ['0', '1', '4'],
 ['0', '1', '4'],
 ['0', '1', '4'],
 ['0', '1', '4'],
 ['0', '1', '2', '4'],
 ['0', '1', '2', '4'],
 ['0', '1', '2'],
 ['0', '1', '5'],
 ['0', '1', '2'],
 ['0', '1', '6'],
 ['0', '1', '6'],
 ['0', '1', '6'],
 ['0', '1', '2'],
 ['0', '1', '2', '6']]

In [159]:
f1_scores(gt, preds)

F1-Macro Score: 0.6974358974358974
F1-Micro Score: 0.8340807174887893


In [136]:
np.argsort(class_weights), np.argmax(np.diff(np.sort(class_weights))) + 1

(array([1, 2, 3, 4, 0]), 4)

In [99]:
for id, label in enumerate(cosine_similarity_embeddings(paper_embs, class_embs).argmax(axis=1)):
    print(id, label, collection[id].gold)

0 0 ['0', '1', '2']
1 0 ['0', '1', '2']
2 0 ['0', '1', '2', '3', '4']
3 0 ['0', '1', '2', '3']
4 0 ['0', '1', '2']
5 0 ['0', '1', '2', '3', '4']
6 0 ['0', '1', '2']
7 0 ['0', '1', '2', '3', '4']
8 0 ['0', '1', '3']
9 1 ['0', '1', '3']
10 0 ['0', '1', '3']
11 0 ['0', '1', '3']
12 0 ['0', '1', '3', '4']
13 0 ['0', '1', '3', '4']
14 0 ['0', '1', '3']
15 1 ['0', '1', '3']
16 0 ['0', '1', '3']
17 2 ['0', '1', '4']
18 2 ['0', '1', '4']
19 2 ['0', '1', '4']
20 2 ['0', '1', '4']
21 2 ['0', '1', '4']
22 2 ['0', '1', '4', '6']
23 2 ['0', '1', '4']
24 2 ['0', '1', '4']
25 2 ['0', '1', '4']
26 0 ['0', '1', '5']
27 3 ['0', '1', '5']
28 0 ['0', '1', '6']
29 4 ['0', '1', '6']
30 4 ['0', '1', '6']
31 4 ['0', '1', '6']
32 0 ['0', '1', '6']
33 4 ['0', '1', '6']


In [79]:
taxo.label2id

{'generative_knowledge_graph_construction': '0',
 'generation_tasks': '1',
 'named_entity_recognition': '2',
 'relation_extraction': '3',
 'event_extraction': '4',
 'entity_linking': '5',
 'knowledge_graph_completion': '6'}

In [83]:
collection[0].gold

['0', '1', '2']

In [87]:
for k, v in psuedo_labels.items():
    print(k, [k in p.gold for p in v])

1 [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
2 [False, True, False, True, False, False, True, False, False, True]
3 [True, True]
4 [True, True, True, True, True]
5 []
6 [True, True, True]


In [38]:
def rankSentences(taxo, sents, sent_phrases, phrase_embs, sent_embs, term_to_idx, bm_score, thresh=3, min_freq=3, percentile=0.999, classify=True):
    
    node_text_ranks = []

    for node_id in tqdm(np.arange(0, len(taxo.label2id))):
        # gather node and its siblings
        focus_node = taxo.root.findChild(str(node_id))
        sibling_nodes = taxo.get_sib(focus_node.node_id, granularity='emb')

        # get phrases of node and its siblings
        focus_phrase = focus_node.getAllTerms(granularity='phrases', children=False)
        focus_phrase_embs = np.stack([taxo.vocab['phrases'][w] for w in focus_phrase], axis=0) # P x dim
        sibling_phrase = [sib.getAllTerms(granularity='phrases', children=False) for sib in sibling_nodes] # list of list of sibling node terms
        sib_phrase_embs = [np.stack([taxo.vocab['phrases'][p] for p in t], axis=0) for t in sibling_phrase] # [(T x dim), ... ()] 
        # get sentences of node and its siblings
        focus_sent = focus_node.getAllTerms(granularity='sentences', children=False)
        focus_sent_embs = np.stack([taxo.vocab['sentences'][s] for s in focus_sent], axis=0) # sentences x dim
        sibling_sent = [sib.getAllTerms(granularity='sentences', children=False) for sib in sibling_nodes] # list [(list of sibling node i terms), ...]
        sib_sent_embs = [np.stack([taxo.vocab['sentences'][s] for s in t], axis=0) for t in sibling_sent] # list [(# of sib sents x dim)]

        # compute target phrase/sentence semantic similarity
        focus_phrase_sim = np.stack([cosine_similarity_embeddings(s, focus_phrase_embs).max(axis=0) for s in phrase_embs], axis=0) # S x [P x N] -> S x N
        avg_focus_phrase_sim = average_with_harmonic_series(focus_phrase_sim, axis=1)  # S x 1

        focus_sent_sim = cosine_similarity_embeddings(sent_embs, focus_sent_embs) # S x N
        avg_focus_sent_sim = average_with_harmonic_series(focus_sent_sim, axis=1)  # S x 1

        # compute sibling sentence semantic dissimilarity
        sib_phrase_sims = [np.stack([cosine_similarity_embeddings(sent_phrase_emb, s_emb).max(axis=0) for sent_phrase_emb in phrase_embs], axis=0) for s_emb in sib_phrase_embs] # siblings x sentences x P x N -> sib x sentences x N
        sib_sent_sims = [cosine_similarity_embeddings(sent_embs, s_emb) for s_emb in sib_sent_embs] # siblings x sentences x sib_sents
        if len(sibling_nodes):
            avg_sib_phrase_sim = np.stack([average_with_harmonic_series(sib_sim, axis=1) for sib_sim in sib_phrase_sims], axis=-1).max(axis=1) # sentences x 1
            avg_sib_sent_sim = np.stack([average_with_harmonic_series(sib_sim, axis=1) for sib_sim in sib_sent_sims], axis=-1).max(axis=1) # sentences x 1
        else:
            avg_sib_phrase_sim = np.zeros_like(avg_focus_phrase_sim)
            avg_sib_sent_sim = np.zeros_like(avg_focus_sent_sim)

        # compute semantic rank
        target_sim_phrase_rank = {idx:rank for rank, idx in enumerate((avg_focus_phrase_sim-avg_sib_phrase_sim).argsort()[::-1])}
        target_sim_sent_rank = {idx:rank for rank, idx in enumerate((avg_focus_sent_sim-avg_sib_sent_sim).argsort()[::-1])}

        # compute target co-occurrence
        target_co_occurrence = np.array([average_with_harmonic_series(getBM25(sent, focus_phrase, term_to_idx, bm_score).mean(axis=0)) for sent in sent_phrases]) # S x 1
        
        # compute sibling co-occurrence
        if len(sibling_nodes):
            sib_co_occurrence = np.array([max([average_with_harmonic_series(getBM25(sent, sib_terms, term_to_idx, bm_score).mean(axis=0)) for sib_terms in sibling_phrase]) for sent in sent_phrases]) # S x 1
        else:
            sib_co_occurrence = np.zeros_like(target_co_occurrence)
        
        # compute co-occurrence rank
        target_co_rank = {idx:rank for rank, idx in enumerate((target_co_occurrence-sib_co_occurrence).argsort()[::-1])}

        joint_rank = compute_joint_ranking([target_sim_phrase_rank, target_sim_sent_rank, target_co_rank]) # idx: rank
        sorted_ranks = sorted(joint_rank.items(), key=lambda x: x[1])

        final_ranks = {}
        for idx, rank in sorted_ranks:
            # if rank > (1-percentile)*len(sents):
            #     break
            if (len(sent_phrases[idx]) > 5): # and (avg_focus_phrase_sim[idx] > avg_sib_phrase_sim[idx]) and (avg_focus_sent_sim[idx] > avg_sib_sent_sim[idx]) and (target_co_occurrence[idx] > sib_co_occurrence[idx]):
                final_ranks[rank] = (sents[idx], avg_focus_phrase_sim[idx], avg_focus_sent_sim[idx], target_co_occurrence[idx])
                # focus_node.internal['sentences'].append(text[idx])
        
        node_text_ranks.append(final_ranks)

    return node_text_ranks

In [29]:
node_id = 4
taxo.root.findChild(str(node_id)), taxo.root.findChild(str(node_id)).external['phrases']
# taxo.root.findChild(str(node_id)).getAllTerms(granularity='phrases', children=False)

(event_extraction,
 ['event_level',
  'event_',
  'event_specific',
  'event_type',
  'event_role',
  'event_generation',
  'event_types',
  'event_extraction',
  'event_arguments',
  'multi_event',
  'event_detection',
  'event_argument_extraction',
  'event_record',
  'event_related',
  'event_masked',
  'event_records',
  'event_relevant',
  'event_correlation'])

In [ ]:
queue = deque([taxo.root])
taxo.root.papers = {paper.id:paper for paper in collection}

while queue:
    focus_node = queue.popleft()
    sibling_nodes = taxo.get_sib(focus_node.node_id, granularity='nodes')

    # get phrases of node and its siblings
    focus_phrase = focus_node.getAllTerms(granularity='phrases', children=False)
    focus_phrase_embs = np.stack([taxo.vocab['phrases'][w] for w in focus_phrase], axis=0) # P x dim
    sibling_phrase = [sib.getAllTerms(granularity='phrases', children=False) for sib in sibling_nodes] # list of list of sibling node terms
    sib_phrase_embs = [np.stack([taxo.vocab['phrases'][p] for p in t], axis=0) for t in sibling_phrase] # [(T x dim), ... ()]
    
    # get sentences of node and its siblings
    focus_sent = focus_node.getAllTerms(granularity='sentences', children=False)
    focus_sent_embs = np.stack([taxo.vocab['sentences'][s] for s in focus_sent], axis=0) # sentences x dim
    sibling_sent = [sib.getAllTerms(granularity='sentences', children=False) for sib in sibling_nodes] # list [(list of sibling node i terms), ...]
    sib_sent_embs = [np.stack([taxo.vocab['sentences'][s] for s in t], axis=0) for t in sibling_sent] # list [(# of sib sents x dim)]


In [ ]:
for paper in tqdm(collection):
    # for each sentence in paper
    ## weigh each phrase in sentence based on how discriminative it is
    for sent in paper.phrase_tokenize:
        
        phrase_sim = cosine_similarity_embeddings()

In [48]:
for p in zip(gt, preds):
    print(f'preds: {sorted(p[1])}; gt: {sorted(p[0])}')

preds: ['0', '1', '2']; gt: ['0', '1', '2']
preds: ['0', '1', '2']; gt: ['0', '1', '2']
preds: ['0', '1', '2', '3', '4']; gt: ['0', '1', '2', '3', '4']
preds: ['0', '1', '2', '3', '4']; gt: ['0', '1', '2', '3']
preds: ['0', '1', '2']; gt: ['0', '1', '2']
preds: ['0', '1', '2', '3', '4']; gt: ['0', '1', '2', '3', '4']
preds: ['0', '1', '2']; gt: ['0', '1', '2']
preds: ['0', '1', '3', '4']; gt: ['0', '1', '2', '3', '4']
preds: ['0', '1', '3', '4']; gt: ['0', '1', '3']
preds: ['0', '1', '3']; gt: ['0', '1', '3']
preds: ['0', '1', '2', '3']; gt: ['0', '1', '3']
preds: ['0', '1', '3']; gt: ['0', '1', '3']
preds: ['0', '1', '2', '3', '4']; gt: ['0', '1', '3', '4']
preds: ['0', '1', '2', '3']; gt: ['0', '1', '3', '4']
preds: ['0', '1', '2', '3', '4']; gt: ['0', '1', '3']
preds: ['0', '1', '2', '3']; gt: ['0', '1', '3']
preds: ['0', '1', '2', '3']; gt: ['0', '1', '3']
preds: ['0', '1', '3', '4']; gt: ['0', '1', '4']
preds: ['0', '1', '4']; gt: ['0', '1', '4']
preds: ['0', '1', '4']; gt: ['0', 

In [202]:
# sentence_pool = internal_sentences
# sentence_pool_emb = np.array([taxo.vocab['sentences'][s] for s in sentence_pool])

In [39]:
# node_external_sent_ranks = expandDiscriminative(taxo, sentence_pool, sentence_pool_emb, granularity='sentences', classify=False, internal=-1)
node_internal_sent_ranks = rankSentences(taxo, sentence_pool, sentence_phrase_pool, phrase_pool_emb, sentence_pool_emb, term_to_idx, bm_score)

100%|██████████| 7/7 [02:28<00:00, 21.24s/it]


In [96]:
taxo.label2id

{'generative_knowledge_graph_construction': '0',
 'generation_tasks': '1',
 'named_entity_recognition': '2',
 'relation_extraction': '3',
 'event_extraction': '4',
 'entity_linking': '5',
 'knowledge_graph_completion': '6'}

In [58]:
for k,v in node_internal_sent_ranks[4].items():
    print(k, v)
    if k > 100:
        break

1 ('[ 8 ] propose a dynamic multi_pooling convolutional neural_network ( dm_ cnn ) , which uses a dynamic multi_pooling layer according to event triggers and arguments for event_extraction', 0.913306785813715, 0.7770894100358857, 0.0031051727799206123)
3 ('the argument part is specic to event_type ei', 0.914617693454433, 0.7869675802903486, 0.002727315162633439)
5 ('the second group contains generation_based event_extraction methods', 0.9080140282650968, 0.8621444665243786, 0.0024849446481576497)
6 ('paper_title : text2event : controllable sequence_to_structure generation for end_to_end event_extraction ; paper_abstract : event_extraction is challenging due to the complex structure of event_records and the semantic gap between text and event', 0.9132553905853479, 0.8347001638762626, 0.0025721657225129344)
7 (', text2event : controllable sequence_to_structure generation for end_to_end event_extraction , in proc', 0.9081088541185993, 0.814699993799282, 0.0028579141235849856)
8 ('2 t ext2

In [41]:
taxo.resetTaxo()

In [42]:
queue = deque([taxo.root])

In [43]:
while queue:
    curr_node = queue.popleft()
    print(f"PROCESSING {curr_node.label}; remaining in queue: {queue}")

    # update vocab with label and description
    taxo.updateVocab([curr_node.label] + curr_node.description.split(), 'phrases')
    taxo.updateVocab(curr_node.description, 'sentences')
    taxo.updateVocab(curr_node.label + " : " + curr_node.description, 'sentences')

    # get common-sense class embedding
    print('before:', curr_node.common_sense['phrases'])
    top_common_phrases, common_phrase_node_emb = computeClassEmb(curr_node, taxo, granularity='phrases', out_phrases=True)
    top_common_sentences, common_sent_node_emb = computeClassEmb(curr_node, taxo, granularity='sentences', out_phrases=True)

    print('after:', top_common_phrases)
    # curr_node.common_sense['phrases'] = top_common_phrases
    # curr_node.common_sense['sentences'] = top_common_sentences

    for child in curr_node.children:
        queue.append(child)

PROCESSING generative_knowledge_graph_construction; remaining in queue: deque([])
before: ['graph_construction', 'knowledge_graph_building', 'generative_knowledge_graph', 'graph_generation', 'graph_building', 'graph_construction_tasks', 'generative_knowledge', 'graph_construction_techniques', 'knowledge_graph_construction', 'graph_building_tasks', 'graph_construction_methods', 'generative_knowledge_graph_construction_techniques', 'graph_construction_approaches', 'knowledge_graph_generation']
after: ['generative_knowledge_graph_construction', 'generative_knowledge_graph_construction_techniques', 'generative_knowledge_graph', 'generative_knowledge', 'knowledge_graph_generation', 'knowledge_graph_construction', 'knowledge_graph_building', 'graph_generation', 'graph_construction_tasks', 'graph_construction_approaches', 'graph_construction', 'graph_construction_techniques', 'graph_construction_methods', 'graph_building', 'graph_building_tasks']
PROCESSING generation tasks; remaining in queu

In [66]:
paper_map = {n.node_id:[] for n in taxo.root.children[0].children}

for p in collection + external_collection:
    for n in taxo.root.children[0].children:
        if any([term in p.vocabulary for term in n.common_sense['phrases']]):
            paper_map[n.node_id].append(p)

In [67]:
for n_id, paper_list in paper_map.items():
    print(n_id, len(paper_list))

2 235
3 56
4 27
5 115
6 119


In [63]:
paper_map["4"]

[id: 2; title: structured_prediction as translation between augmented natural languages; abstract: we propose a new framework , translation between augmented natural languages ( tanl ) , to solve many structured_prediction language tasks including joint_entity_and_relation_extraction , nested named_entity recognition , relation_classification , semantic_role_labeling , event_extraction , coreference_resolution , and dialogue_state_tracking . instead of tackling the problem by training task_specific discriminative classifiers , we frame it as a translation task between augmented natural languages , from which the task_relevant information can be easily extracted . our approach can match or outperform task_specific models on all tasks , and in particular , achieves new state_of_the_art results on joint_entity_and_relation_extraction ( conll04 , ade , nyt , and ace2005 datasets ) , relation_classification ( fewrel and tacred ) , and semantic_role_labeling ( conll_2005 and conll_2012 ) . w

In [187]:
from main import rankPhrases

In [206]:
ranked_external, class_emb = rankPhrases(external_sentences, taxo.root, taxo, use_class_emb=True, granularity='sentences', out_phrases=True)

In [33]:
queue = deque([taxo.root])

In [34]:
while queue:
    curr_node = queue.popleft()
    parent_node = curr_node.parents[0]

    print(f"PROCESSING {curr_node.label}; remaining in queue: {queue}")

    # EXTERNAL: get partitioned set of external phrases and sentences
    curr_node.external['phrases'] = external_phrases # parent_node.external['phrases']
    curr_node.external['sentences'] = external_sentences # parent_node.external['sentences']
  
    # get top external phrases & external sentences
    top_phrases, external_phrase_class_emb = computeClassEmb(curr_node, taxo, class_emb=True, granularity='phrases', out_phrases=True)
    top_sentences, external_sentence_class_emb = computeClassEmb(curr_node, taxo, class_emb=True, granularity='sentences', out_phrases=True)

    top_external_phrases = [phrase for phrase in top_phrases if phrase in curr_node.external['phrases']]
    top_external_sentences = [sent for sent in top_sentences if sent in curr_node.external['sentences']]
    curr_node.external['phrases'] = top_external_phrases
    curr_node.external['sentences'] = top_external_sentences

    # INTERNAL: get partitioned set of internal phrases and sentences
    curr_node.internal['phrases'] = internal_phrases # parent_node.internal['phrases']
    curr_node.internal['sentences'] = internal_sentences # parent_node.internal['sentences']

    # get top internal phrases & sentences
    top_phrases, internal_phrase_class_emb = computeClassEmb(curr_node, taxo, class_emb=True, granularity='phrases', out_phrases=True)
    top_sentences, internal_sentence_class_emb = computeClassEmb(curr_node, taxo, class_emb=True, granularity='sentences', out_phrases=True)

    top_internal_phrases = [phrase for phrase in top_phrases if phrase in curr_node.internal['phrases']]
    top_internal_sentences = [sent for sent in top_sentences if sent in curr_node.internal['sentences']]
    curr_node.internal['phrases'] = top_internal_phrases
    curr_node.internal['sentences'] = top_internal_sentences

    for child in curr_node.children:
        queue.append(child)

PROCESSING generative_knowledge_graph_construction; remaining in queue: deque([])
PROCESSING generation tasks; remaining in queue: deque([])
PROCESSING named_entity_recognition; remaining in queue: deque([relation_extraction, event_extraction, entity_linking, knowledge_graph_completion])
PROCESSING relation_extraction; remaining in queue: deque([event_extraction, entity_linking, knowledge_graph_completion])
PROCESSING event_extraction; remaining in queue: deque([entity_linking, knowledge_graph_completion])
PROCESSING entity_linking; remaining in queue: deque([knowledge_graph_completion])
PROCESSING knowledge_graph_completion; remaining in queue: deque([])


In [40]:
taxo.root.children[0].children[0].external

{'phrases': ['entity_recognition',
  'bag_of_entity',
  'bag_of_entities',
  'named',
  'anti_nose',
  'tokens',
  'artificial_neural_networks',
  'facial_expression',
  '//github.com/wjn1996/mathematical_knowledge_entity_recognition',
  'artificial_intelligence',
  'andrecognize',
  'token',
  'murder',
  'tokenization',
  'negated',
  'mention_pronoun',
  'word_vector',
  'weaponry',
  'inflated',
  'optical_character_recognition',
  'token_based',
  "'phoneme",
  'word_character',
  'natural_language',
  'intent_classifier',
  'entity_classification',
  'entity_detection',
  'mention',
  'name',
  'non_named',
  'word_by_word',
  'weapon',
  'named_entity_extraction',
  'noun',
  'sign_language',
  'applications_of_artificial_intelligence',
  'noun_phrase',
  'intelligent_agent',
  'pronoun',
  'character_vocabulary',
  'word_entity',
  'support_vector_machine',
  'utterances',
  'automatic_speech_recognition',
  'naming',
  'chinese_language',
  'uttered',
  'controlled_vocabulary'

In [265]:
ranked_papers = taxo.rankPapers([curr_node.emb['sentences'] for curr_node in taxo.root.children], 2, internal=True, phrase=False, top_k=20)

In [266]:
ranked_papers

[(66, 0),
 (65, 1),
 (71, 2),
 (69, 3),
 (51, 4),
 (47, 5),
 (55, 6),
 (50, 7),
 (48, 8),
 (70, 9),
 (24, 10),
 (54, 11),
 (4, 12),
 (62, 13),
 (25, 14),
 (11, 15),
 (22, 16),
 (53, 17),
 (67, 18),
 (59, 19)]

In [286]:
paper_embs = {idx: paper_emb for idx, paper_emb in enumerate(sentence_model.encode([paper.title for paper in collection]))}

In [306]:
queue = deque([taxo.root])

In [307]:
class_labels = {}
class_mappings = {}
paper_scores = {}

while queue:
    target_node = queue.popleft()
    print(f"PROCESSING {target_node.label}; remaining in queue: {queue}")

    class_ids = [child.node_id for child in target_node.children]
    print(class_ids)

    if len(target_node.children) <= 1:
        continue
    
    ranked_papers = []
    for idx in np.arange(len(class_ids)):
        ranked_papers.append(taxo.rankPapers([curr_node.emb['sentences'] for curr_node in target_node.children], idx, internal=True, phrase=False, top_k=20))
    
    for p in ranked_papers:
        print("next!")
        for c in p:
            print(collection[c[0]].title)

    ranked_paper_embs = [average_with_harmonic_series([paper_embs[p[0]] for p in c]) for c in ranked_papers]


    node_labels, node_paper_scores, node_mapping = taxo.mapPapers(np.array(list(paper_embs.values())), target_node.children, ranked_paper_embs)

    class_labels[target_node.node_id] = node_labels
    paper_scores[target_node.node_id] = node_paper_scores
    class_mappings[target_node.node_id] = node_mapping


    # full_text_sim = cosine_similarity_embeddings(list(paper_embs.values()), ranked_paper_embs)
    # full_text_preds = full_text_sim.argmax(axis=1)
    # full_text_scores = [p_id in target_node.children[pred].gold for p_id, pred in enumerate(full_text_preds)]

    # for idx, paper_id in enumerate(target_node.papers):
    #     target_node.children[winner_idxs[idx]].papers[paper_id] = target_node.papers[paper_id]

    print("ranking-based classification", sum(full_text_scores)/len(full_text_scores))

    for child in target_node.children:
        queue.append(child)

PROCESSING using_llms_on_graphs; remaining in queue: deque([])
['1', '2', '3']
next!
node feature_extraction by self_supervised multi_scale neighborhood_prediction
chemformer : a pre_trained transformer for computational_chemistry
catalyst property_prediction with catberta : unveiling feature exploration strategies through large_language_models
efficient and effective training of language and graph_neural_network models
regression transformer enables concurrent sequence regression and generation for molecular language_modelling
graph_toolformer : to empower llms with graph_reasoning ability via prompt augmented by chatgpt
llm4dyg : can large_language_models solve spatial_temporal problems on dynamic_graphs ?
label_free node_classification on graphs with large_language_models ( llms )
can language_models solve_graph_problems in natural_language ?
relm : leveraging language_models for enhanced chemical_reaction prediction
text2mol : cross_modal molecule retrieval with natural_language qu

In [ ]:
collection[0].rankSentences()

In [308]:
class_labels

{'0': [[1],
  [0, 1],
  [1, 0],
  [0],
  [1],
  [0, 1],
  [1],
  [0, 1],
  [0],
  [],
  [1],
  [1],
  [1],
  [1],
  [0],
  [0, 1],
  [1],
  [1],
  [1],
  [1],
  [1],
  [1],
  [0, 1],
  [1],
  [0],
  [],
  [1],
  [],
  [0],
  [1],
  [1],
  [0, 1],
  [0],
  [0, 1],
  [1],
  [0, 1],
  [1, 0],
  [1],
  [1],
  [1],
  [0, 1],
  [0, 1],
  [0, 1],
  [0, 1],
  [0, 1],
  [1],
  [0],
  [],
  [0],
  [],
  [],
  [],
  [1],
  [0],
  [],
  [],
  [1],
  [0],
  [],
  [0],
  [0],
  [0],
  [],
  [1],
  [],
  [2],
  [2],
  [],
  [0],
  [],
  [],
  [2]],
 '2': [[1, 2],
  [2],
  [2],
  [2],
  [2],
  [1, 2],
  [1, 2],
  [2],
  [2],
  [],
  [1, 2],
  [2, 1],
  [2, 1],
  [1, 2],
  [2],
  [1, 2],
  [2, 1],
  [2, 1],
  [0, 1],
  [1],
  [1, 0],
  [1],
  [0, 1],
  [1],
  [0],
  [],
  [0],
  [0],
  [],
  [],
  [],
  [2, 1],
  [0],
  [2, 1],
  [1],
  [1, 2],
  [2],
  [1],
  [1],
  [1],
  [0],
  [2],
  [0, 1],
  [1],
  [0, 1],
  [1],
  [0],
  [2],
  [2, 0],
  [],
  [],
  [0],
  [1, 0],
  [2],
  [1, 0],
  [],
  [0],
 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0])

In [65]:
from utils import rank_by_class_discriminative_significance, filter_by_class_discriminative_significance

In [97]:
phrase_reprs = np.concatenate([taxo.static_emb[w].reshape((-1, 768)) for w in internal_phrases], axis=0)
ranks = rank_by_class_discriminative_significance(phrase_reprs, [average_with_harmonic_series([taxo.static_emb[w] for w in c.external['phrases']]) for c in focus_node.parents[0].children], focus_node.parents[0].children.index(focus_node))
ranked_tok = {internal_phrases[idx]:rank for idx, rank in ranks.items()}

ZeroDivisionError: Weights sum to zero, can't be normalized

In [98]:
ranked_tok

{'learning': 0,
 'reasoning': 1,
 'classification': 2,
 'retrieval': 3,
 'inference': 4,
 'smoothing': 5,
 'graph': 6,
 'model': 7,
 'knowledge': 8,
 'representations': 9,
 'policy': 10,
 'representation': 11,
 'based': 12,
 'understanding': 13,
 'cross': 14,
 'analysis': 15,
 'models': 16,
 'representation_learning': 17,
 'taxonomy': 18,
 'routing': 19,
 'function': 20,
 'generation': 21,
 'grouping': 22,
 'encoding': 23,
 'network': 24,
 'trajectory': 25,
 'dialogue': 26,
 'dictionary': 27,
 'using': 28,
 'discourse': 29,
 'algorithm': 30,
 'derivation': 31,
 'search': 32,
 'with': 33,
 'estimates': 34,
 'modeling': 35,
 'attention': 36,
 'prompting': 37,
 'targets': 38,
 'molecular': 39,
 'rating': 40,
 'programming': 41,
 'scene': 42,
 'transmission': 43,
 'feature': 44,
 'information': 45,
 'for': 46,
 'task': 47,
 'semantic': 48,
 'collaboration': 49,
 'hierarchy': 50,
 'conditioned': 51,
 'reward': 52,
 'action': 53,
 'correction': 54,
 'searches': 55,
 'structure': 56,
 'orderi

In [96]:
focus_node = taxo.root.findChild("18")
focus_node.label

'data_augmentation'

In [36]:
focus_node.external['phrases']

['answer_set',
 'answer_prediction',
 'answer_generation',
 'invoices',
 'answering_systems',
 'image_question',
 'answer_candidates',
 'answer_answer',
 'disengagement',
 'conversation_generation',
 'answer_contained',
 'chatting',
 'non_mediated',
 'present_participle',
 'past_participle',
 'soliciting',
 'dialogue_generation',
 'intolerant',
 'donations',
 'candidate_answers',
 'hate_speech',
 'self_distillation',
 'dialogue_interaction',
 'teacher_forcing',
 'know_why',
 'exclamation',
 'sub_questions',
 'canencourage',
 'question_comment',
 'complex_questions',
 'never_before_seen',
 'question2subgraph',
 'redirected',
 'help_seekers',
 'query_answering',
 'unanswerable',
 'frustratingly',
 'quests',
 'hate_keyword',
 'dialogue_management',
 'multi_teacher',
 'aroused',
 'non_directly',
 'few/zero_shot',
 'weakly_annotated',
 'arouse',
 'answering_complex',
 'dining',
 'helpdesks',
 'missing_links',
 'redirection',
 'hop_attention',
 'audiences',
 'delegate',
 'conversational_agen

In [30]:
paper_embs = {idx: paper_emb for idx, paper_emb in enumerate(sentence_model.encode([paper.title for paper in collection]))}

In [31]:
queue = deque([taxo.root])

In [32]:
while queue:
    target_node = queue.popleft()
    print(f"PROCESSING {target_node.label}; remaining in queue: {queue}")

    class_ids = [child.node_id for child in target_node.children]
    print(class_ids)

    phrase_class_embs = [computeClassEmb(curr_node, taxo, granularity='phrases') for curr_node in target_node.children]
    sent_class_embs = [computeClassEmb(curr_node, taxo, granularity='sentences') for curr_node in target_node.children]
    joint_class_embs = [(p+s)/2 for p, s in zip(phrase_class_embs, sent_class_embs)]

    encoded_papers = np.array([paper_embs[p_id] for p_id in target_node.papers])
    phrase_sim = cosine_similarity_embeddings(encoded_papers, phrase_class_embs) # P x C
    sent_sim = cosine_similarity_embeddings(encoded_papers, sent_class_embs)
    joint_sim = cosine_similarity_embeddings(encoded_papers, joint_class_embs)

    phrase_winner = phrase_sim.argmax(axis=1) # P x 1
    sent_winner = sent_sim.argmax(axis=1)
    joint_winner = joint_sim.argmax(axis=1)
    winner_idxs = stats.mode(np.stack((phrase_winner, sent_winner, joint_winner), axis=1), axis=1, keepdims=False)[0]
    labels = [class_ids[winner_idx] for winner_idx in winner_idxs]
    correct = [paper_id in target_node.children[winner_idxs[idx]].gold for idx, paper_id in enumerate(target_node.papers)]
    print("before ranking-based classification: ", sum(correct)/len(correct))

    scores = (phrase_sim[np.arange(len(phrase_sim)), 
                        winner_idxs] + sent_sim[np.arange(len(sent_sim)), 
                                                winner_idxs] + joint_sim[np.arange(len(joint_sim)), 
                                                                        winner_idxs])/3

    class_map = {i:[] for i in class_ids}
    for idx, paper_id in enumerate(target_node.papers):
        class_map[labels[idx]].append((scores[idx], target_node.papers[paper_id]))

    class_map = {i:sorted(class_map[i], key=lambda x: -x[0]) for i in class_ids}
    full_text_class_embs = []
    for c_id in tqdm(class_ids):
        full_text_class_embs.append(average_with_harmonic_series(np.array([paper_embs[p[1].id] for p in class_map[c_id]])))

    full_text_sim = cosine_similarity_embeddings(np.array([paper_embs[p_id] for p_id in target_node.papers]), full_text_class_embs)
    full_text_preds = full_text_sim.argmax(axis=1)
    full_text_scores = [p_id in target_node.children[pred].gold for p_id, pred in enumerate(full_text_preds)]

    for idx, paper_id in enumerate(target_node.papers):
        target_node.children[winner_idxs[idx]].papers[paper_id] = target_node.papers[paper_id]

    print("ranking-based classification", sum(full_text_scores)/len(full_text_scores))

    for child in target_node.children:
        queue.append(child)

PROCESSING using_llms_on_graphs; remaining in queue: deque([])
['1', '2', '3']
before ranking-based classification:  0.6111111111111112


100%|██████████| 3/3 [00:00<00:00, 4096.00it/s]

ranking-based classification 0.7916666666666666
PROCESSING pure_graphs; remaining in queue: deque([text_rich_graphs, text_paired_graphs])
['4']


before ranking-based classification:  0.35714285714285715


100%|██████████| 1/1 [00:00<00:00, 2270.87it/s]

...

ranking-based classification 0.0
PROCESSING llm_as_aligner; remaining in queue: deque([llm_as_predictor, llm_as_aligner, direct_answering, heuristic_reasoning, algorithmic_reasoning, graph_as_sequence, graph_empowered_llm, graph_aware_llm_finetuning, optimization, data_augmentation, knowledge_distillation])
['21', '22']
before ranking-based classification:  0.0


100%|██████████| 2/2 [00:00<00:00, 4718.00it/s]


ranking-based classification 0.0
PROCESSING llm_as_predictor; remaining in queue: deque([llm_as_aligner, direct_answering, heuristic_reasoning, algorithmic_reasoning, graph_as_sequence, graph_empowered_llm, graph_aware_llm_finetuning, optimization, data_augmentation, knowledge_distillation, prediction_alignment, latent_space_alignment])
['24', '25']
before ranking-based classification:  0.0


  0%|          | 0/2 [00:00<?, ?it/s]


ZeroDivisionError: Weights sum to zero, can't be normalized

In [52]:
[p[1].id for p in class_map[c_id]]

[8, 2, 7, 3, 59, 49, 47]

In [ ]:
full_text_class_embs.append(average_with_harmonic_series(np.array([encoded_papers[p[1].id] for p in class_map[c_id]])))

In [36]:
target_node = taxo.root#.children[1]

class_ids = [child.node_id for child in target_node.children]
print(class_ids)

phrase_class_embs = [computeClassEmb(curr_node, taxo, granularity='phrases') for curr_node in target_node.children]
sent_class_embs = [computeClassEmb(curr_node, taxo, granularity='sentences') for curr_node in target_node.children]
# joint_class_embs = [computeClassEmb(curr_node, taxo, granularity='mixed') for curr_node in target_node.children]
joint_class_embs = [(p+s)/2 for p, s in zip(phrase_class_embs, sent_class_embs)]

['8', '14', '20']
llm_as_predictor: 57 phrases filtered!
llm_as_encoder: 75 phrases filtered!
llm_as_aligner: 27 phrases filtered!
llm_as_predictor: 46 sentences filtered!
llm_as_encoder: 12 sentences filtered!
llm_as_aligner: 18 sentences filtered!


In [18]:
external_phrase_embs, sim_diff, keep_phrase = compareClasses(external_phrases, taxo, target_node.children[0], 'phrases')
filtered_embs = external_phrase_embs[keep_phrase, :]
filtered_external_phrases = list(compress(external_phrases, keep_phrase))
filtered_diffs = sim_diff[keep_phrase]
print(f'{target_node.label}: {len(external_phrases) - len(filtered_external_phrases)} phrases filtered!')

text_rich_graphs: 6586 phrases filtered!


In [14]:
encoded_papers = sentence_model.encode([collection[p_id].raw_text for p_id in target_node.papers])
phrase_sim = cosine_similarity_embeddings(encoded_papers, phrase_class_embs) # P x C
sent_sim = cosine_similarity_embeddings(encoded_papers, sent_class_embs)
joint_sim = cosine_similarity_embeddings(encoded_papers, joint_class_embs)

phrase_winner = phrase_sim.argmax(axis=1) # P x 1
sent_winner = sent_sim.argmax(axis=1)
joint_winner = joint_sim.argmax(axis=1)
winner_idxs = stats.mode(np.stack((phrase_winner, sent_winner, joint_winner), axis=1), axis=1, keepdims=False)[0]
labels = [class_ids[winner_idx] for winner_idx in winner_idxs]
correct = [paper_id in target_node.children[winner_idxs[idx]].gold for idx, paper_id in enumerate(target_node.papers)]
print("before ranking-based classification: ", sum(correct)/len(correct))

scores = (phrase_sim[np.arange(len(phrase_sim)), 
                    winner_idxs] + sent_sim[np.arange(len(sent_sim)), 
                                            winner_idxs] + joint_sim[np.arange(len(joint_sim)), 
                                                                      winner_idxs])/3

class_map = {i:[] for i in class_ids}
for idx, paper_id in enumerate(target_node.papers):
    class_map[labels[idx]].append((scores[idx], target_node.papers[paper_id]))

class_map = {i:sorted(class_map[i], key=lambda x: -x[0]) for i in class_ids}
full_text_class_embs = []
for c_id in tqdm(class_ids):
    full_text_class_embs.append(average_with_harmonic_series(sentence_model.encode([p[1].raw_text for p in class_map[c_id]])))

full_text_sim = cosine_similarity_embeddings(sentence_model.encode([collection[p_id].raw_text for p_id in target_node.papers]), full_text_class_embs)
full_text_preds = full_text_sim.argmax(axis=1)
full_text_scores = [p_id in target_node.children[pred].gold for p_id, pred in enumerate(full_text_preds)]

# for idx, paper_id in enumerate(target_node.papers):
#     target_node.children[winner_idxs[idx]].papers[paper_id] = target_node.papers[paper_id]

print("ranking-based classification", sum(full_text_scores)/len(full_text_scores))

before ranking-based classification:  0.8194444444444444


100%|██████████| 3/3 [00:01<00:00,  2.66it/s]


ranking-based classification 0.9861111111111112


In [ ]:
title: 0.24444444444444444
title + abstract: 0.26666666666666666
raw_text: 0.28888888888888886

In [13]:
os.environ["CUDA_VISIBLE_DEVICES"]="5,6"

In [14]:
device = torch.device("cuda")

In [15]:
tokenizer = BertTokenizer.from_pretrained('/home/pk36/Comparative-Summarization/bert_full_ft/checkpoint-8346/')
model = BertModel.from_pretrained('/home/pk36/Comparative-Summarization/bert_full_ft/checkpoint-8346/', output_hidden_states=True).to(device)

Some weights of BertModel were not initialized from the model checkpoint at /home/pk36/Comparative-Summarization/bert_full_ft/checkpoint-8346/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [310]:
cnt = defaultdict(int)
all_raw_phrases = []
token_lens = {}

for paper in tqdm(collection):
	all_raw_phrases.extend(paper.raw_text.strip().split())

for phrase in tqdm(set(all_raw_phrases)):
	token_lens[phrase] = len(tokenizer(phrase).input_ids)

cnt = Counter(all_raw_phrases)

100%|██████████| 51136/51136 [00:06<00:00, 7733.38it/s]


In [253]:
def computeProp(phrase_sim, sent_sim):
    print('phrase avg:', phrase_sim.mean(axis=0))
    print('sent avg:', sent_sim.mean(axis=0))
    phrase_count = Counter(phrase_sim.argmax(axis=1))
    phrase_list = np.array([phrase_count[i] 
                            if i in phrase_count 
                            else 0 
                            for i in np.arange(len(taxo.root.children))])
    sent_count = Counter(sent_sim.argmax(axis=1))
    sent_list = np.array([sent_count[i] 
                          if i in sent_count 
                          else 0 
                          for i in np.arange(len(taxo.root.children))])
    
    print(f"Phrase: {phrase_list/sum(phrase_list)}")
    print(f"Sentence: {sent_list/sum(sent_list)}")
    return phrase_count, sent_count

In [352]:
def chunkify(text, token_lens, length=512):
	chunks = [[]]
	split_text = text.split()
	count = 0
	for word in split_text:
		new_count = count + token_lens[word] + 2 # 2 for [CLS] and [SEP]
		if new_count > length:
			chunks.append([word])
			count = token_lens[word]
		else:
			chunks[len(chunks) - 1].append(word)
			count = new_count
	
	chunks = [" ".join(chunk) for chunk in chunks]
	return chunks if len(chunks) > 1 else chunks[0]

def encode(w):
	if type(w) == str:
		chunks = chunkify(w, token_lens)
	else:
		chunks = w
	
	encoded_data = tokenizer(chunks, padding=True, return_tensors="pt").to(device)
	with torch.no_grad():
		output = model(**encoded_data)
	
	# Get all hidden states
	if (type(w) == str) and (len(output.last_hidden_state.shape) == 3):
		return output.last_hidden_state.mean(axis=(0,1)).cpu().numpy()
	else:
		return output.last_hidden_state.mean(axis=1).cpu().numpy()

In [106]:
cnt = defaultdict(int)
vocab_emb = {}
vocab_sent_emb = {}

for paper in tqdm(collection):
	data = paper.raw_text.strip().split()
	sent_data = paper.raw_text.strip().split(" . ")
	p_embs = sentence_model.encode(data)
	s_embs = sentence_model.encode(sent_data)
	for w_id, word in enumerate(data):
		cnt[word] += 1
		if word not in vocab_emb:
			vocab_emb[word] = p_embs[w_id]
	for s_id, sent in enumerate(sent_data):
		vocab_sent_emb[sent] = s_embs[s_id]


100%|██████████| 72/72 [05:51<00:00,  4.88s/it]


In [93]:
for w in tqdm(all_common_phrases):
    if w not in vocab_emb:
        vocab_emb[w] = sentence_model.encode(w)

100%|██████████| 423/423 [00:03<00:00, 135.99it/s]


In [100]:
sorted(cnt.items(), key=lambda x: x[1], reverse=True)

[(',', 46604),
 ('.', 31280),
 ('the', 30334),
 ('and', 14675),
 (')', 14552),
 ('(', 14263),
 ('of', 12858),
 ('to', 10450),
 ('in', 10030),
 ('a', 9576),
 ('for', 6881),
 (':', 6764),
 ('we', 6445),
 ('is', 5707),
 ('on', 5222),
 ('as', 4366),
 ('with', 4331),
 ('[', 3910),
 ('graph', 3906),
 (']', 3898),
 ('et', 3473),
 ('that', 3272),
 ('are', 3188),
 (';', 2906),
 ('model', 2896),
 ('al.', 2762),
 ('from', 2608),
 ('this', 2602),
 ('by', 2589),
 ('can', 2191),
 ('1', 1978),
 ('our', 1948),
 ('be', 1889),
 ('%', 1851),
 ('tasks', 1850),
 ('models', 1797),
 ('llms', 1727),
 ('which', 1720),
 ('an', 1640),
 ('text', 1515),
 ('2', 1514),
 ('performance', 1494),
 ('it', 1456),
 ('reasoning', 1379),
 ('information', 1368),
 ('node', 1330),
 ('molecule', 1328),
 ('data', 1311),
 ('nodes', 1249),
 ('language', 1242),
 ('task', 1206),
 ('3', 1181),
 ('each', 1145),
 ('or', 1129),
 ('table', 1107),
 ('results', 1087),
 ('have', 1038),
 ('not', 1023),
 ('dataset', 1015),
 ('paper', 1003),
 (

In [102]:
iv_phrases = [w for w in cnt if (w not in stopwords.words('english'))]
phrase_reprs = np.concatenate([vocab_emb[w].reshape((-1, 768)) for w in iv_phrases], axis=0)

In [108]:
sents = list(vocab_sent_emb.keys())
sent_reprs = np.concatenate([vocab_sent_emb[s].reshape((-1, 768)) for s in sents], axis=0)

In [60]:
if not os.path.exists(f"SeeTopic/{args.dataset}"):
    os.makedirs(f"SeeTopic/{args.dataset}")

with open(f"SeeTopic/{args.dataset}/{args.dataset}.txt", "w") as f:
    for p in taxo.external_collection:
        f.write(f"paper_title : {p.title} ; paper_abstract : {p.abstract}\n")
    for p in taxo.collection:
        f.write(f"paper_title : {p.title} ; paper_abstract : {p.abstract}\n")

children_with_terms = taxo.root.getChildren(terms=True)

with open(f"SeeTopic/{args.dataset}/keywords_0.txt", "w") as f:
    for idx, c in enumerate(children_with_terms):
        str_c = ",".join(c[1])
        f.write(f"{idx}:{c[0]},{str_c}\n")

In [61]:
os.chdir("./SeeTopic")
subprocess.check_call(['./seetopic.sh', args.dataset, str(args.iters), args.model])
os.chdir("../")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


===Get PLM Embeddings===


Some weights of BertModel were not initialized from the model checkpoint at /home/pk36/Comparative-Summarization/bert_full_ft/checkpoint-8346/ and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


####### CONSTRUCTING AND TOKENIZING VOCAB #######
####### COMPUTING STATIC EMBEDDINGS #######
####### COMPUTING RAW WORD EMBEDDINGS #######


100%|██████████| 116/116 [00:30<00:00,  3.79it/s]


===Iter 0: PLM Module===
===Iter 1: PLM Module===
===Iter 1: Local Module===
make: 'cate' is up to date.
Starting training using file ../llm_graph/llm_graph.txt
Reading topics from file llm_graph_1/keywords.txt
Vocab size: 14396
Words in train file: 1290798
Read 3 topics
graph_context	representation_learning	graph_search	
augmentation_methods	alignment_methods	graph_neural_networks	
representation_enhancement	sequence_graph	graph_to_sequence	
Pre-training for 2 epochs, in total 2 + 10 = 12 epochs
Alpha: 0.000099  Progress: 99.67%  Words/thread/sec: 13.26k  Topic mining results written to file llm_graph_1/res_cate.txt
===Iter 1: Ensemble===
===Iter 2: PLM Module===
===Iter 2: Local Module===
make: 'cate' is up to date.
Starting training using file ../llm_graph/llm_graph.txt
Reading topics from file llm_graph_2/keywords.txt
Vocab size: 14396
Words in train file: 1290798
Read 3 topics
graph_context	representation_learning	graph_search	graph_reasoning	answer_generation	graph_generation	
au